# Numerical SOS decomposition

## Preliminaries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ncpol2sdpa as ncp
import json

In [2]:
# Define quantum operators
A_config = [2,2]
B_config = [2,2]

# Operators in problem
A = [Ai for Ai in ncp.generate_measurements(A_config, 'A')]
B = [Bj for Bj in ncp.generate_measurements(B_config, 'B')]

ops = ncp.flatten([A,B])        # Base monomials involved in problem
subs = {}

# 1. Unitarity: A^2 = 1
for op in ops:
    subs[op**2] = 1

# 2. Commutation: force a canonical ordering B*A → A*B
for Ai in ncp.flatten(A):
    for Bj in ncp.flatten(B):
        subs[Bj * Ai] = Ai * Bj

## Test: the CHSH expression

In [3]:
moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = []                       # extra monomials

obj = (A[0][0]*B[0][0] + A[1][0]*B[0][0] + A[0][0]*B[1][0] - A[1][0]*B[1][0])/(2*np.sqrt(2))

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=5,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.primal, -sdp.dual, sdp.status)
#sdp.monomial_index

# Extract primal (moment matrix) and dual (SOS matrix)
Gamma = sdp.x_mat      # Moment matrix
W = sdp.y_mat          # Dual solution — SOS matrix in monomial basis

1.0000000000810259 0.9999999999441891 optimal


## $T_{1 + A + B}$

In [4]:
moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = []                       # extra monomials

# Compute quantum value of Bell expression at (p1, p4) for a given Theta
s = np.sqrt(2)
Theta = np.pi/4
p1 = 1 - 1/np.sqrt(2)
p4 = 0

tan_2T = np.tan(2 * Theta)
sin_2T = np.sin(2 * Theta)
cos_2T = np.cos(2 * Theta)
cos_4T = np.cos(4 * Theta)
sin_4T = np.sin(4 * Theta)

obj = 1/16 * (
        4 * (s * (B[0][0] + B[1][0]) / (tan_2T * sin_2T) + 4 * p1 * A[0][0] + 8 * p4 * A[1][0]) +
        (4 / s) / (sin_2T ** 2) * (
            (B[0][0] + B[1][0]) * (-3 + cos_4T) * p1
            - 2 * A[0][0] * B[0][0] * (cos_4T - p1 * cos_2T)
            - 2 * A[0][0] * B[1][0] * (cos_4T - p1 * cos_2T)
            + 2 * sin_2T * (
                4 * (-B[0][0] + B[1][0]) * p4 * sin_2T ** 2
                + A[1][0] * B[1][0] * (-1 + cos_2T * p1 - 2 * p4 * sin_4T)
                - A[1][0] * B[0][0] * (-1 + cos_2T * p1 + 2 * p4 * sin_4T)
            )
        )
    )

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.primal, -sdp.dual, sdp.status)

sdp.write_to_file("num_level1+A+B.csv")

1.085786437700181 1.0857864365382908 optimal


## $T_{AB} = T_{1 + A + B + AB}$

In [5]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)

    #AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()   # extra monomials

# Compute quantum value of Bell expression at (p1, p4) for a given Theta
s = np.sqrt(2)
Theta = np.pi/4
p1 = 1 - 1/np.sqrt(2)
p4 = 0

tan_2T = np.tan(2 * Theta)
sin_2T = np.sin(2 * Theta)
cos_2T = np.cos(2 * Theta)
cos_4T = np.cos(4 * Theta)
sin_4T = np.sin(4 * Theta)

obj = 1/16 * (
        4 * (s * (B[0][0] + B[1][0]) / (tan_2T * sin_2T) + 4 * p1 * A[0][0] + 8 * p4 * A[1][0]) +
        (4 / s) / (sin_2T ** 2) * (
            (B[0][0] + B[1][0]) * (-3 + cos_4T) * p1
            - 2 * A[0][0] * B[0][0] * (cos_4T - p1 * cos_2T)
            - 2 * A[0][0] * B[1][0] * (cos_4T - p1 * cos_2T)
            + 2 * sin_2T * (
                4 * (-B[0][0] + B[1][0]) * p4 * sin_2T ** 2
                + A[1][0] * B[1][0] * (-1 + cos_2T * p1 - 2 * p4 * sin_4T)
                - A[1][0] * B[0][0] * (-1 + cos_2T * p1 + 2 * p4 * sin_4T)
            )
        )
    )

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.primal, -sdp.dual, sdp.status)

sdp.write_to_file("num_level1+A+B+AB.csv")

1.0091153568571396 1.0091153431363185 optimal


## $T_{1 + A + B + AA' + BB' + AB}$

In [6]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)

    # AA
    for a in Aflat:
        for a2 in Aflat:
            monos += [a*a2]

    # BB
    for b in Bflat:
        for b2 in Bflat:
            monos += [b*b2]
    
    # AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()   # extra monomials

# Compute quantum value of Bell expression at (p1, p4) for a given Theta
s = np.sqrt(2)
Theta = np.pi/4
p1 = 1 - 1/np.sqrt(2)
p4 = 0

tan_2T = np.tan(2 * Theta)
sin_2T = np.sin(2 * Theta)
cos_2T = np.cos(2 * Theta)
cos_4T = np.cos(4 * Theta)
sin_4T = np.sin(4 * Theta)

obj = 1/16 * (
        4 * (s * (B[0][0] + B[1][0]) / (tan_2T * sin_2T) + 4 * p1 * A[0][0] + 8 * p4 * A[1][0]) +
        (4 / s) / (sin_2T ** 2) * (
            (B[0][0] + B[1][0]) * (-3 + cos_4T) * p1
            - 2 * A[0][0] * B[0][0] * (cos_4T - p1 * cos_2T)
            - 2 * A[0][0] * B[1][0] * (cos_4T - p1 * cos_2T)
            + 2 * sin_2T * (
                4 * (-B[0][0] + B[1][0]) * p4 * sin_2T ** 2
                + A[1][0] * B[1][0] * (-1 + cos_2T * p1 - 2 * p4 * sin_4T)
                - A[1][0] * B[0][0] * (-1 + cos_2T * p1 + 2 * p4 * sin_4T)
            )
        )
    )

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.primal, -sdp.dual, sdp.status)
sdp.write_to_file("num_level1+A+B+AA+BB+AB.csv")

1.00032614413522 1.0003261308776583 optimal


## $T_{1+A+B+ABB'}$

In [7]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)

    # ABB
    for a in Aflat:
        for b in Bflat:
            for b2 in Bflat:
                monos += [a*b*b2]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()   # extra monomials

# Compute quantum value of Bell expression at (p1, p4) for a given Theta
s = np.sqrt(2)
Theta = np.pi/4
p1 = 1 - 1/np.sqrt(2)
p4 = 0

tan_2T = np.tan(2 * Theta)
sin_2T = np.sin(2 * Theta)
cos_2T = np.cos(2 * Theta)
cos_4T = np.cos(4 * Theta)
sin_4T = np.sin(4 * Theta)

obj = 1/16 * (
        4 * (s * (B[0][0] + B[1][0]) / (tan_2T * sin_2T) + 4 * p1 * A[0][0] + 8 * p4 * A[1][0]) +
        (4 / s) / (sin_2T ** 2) * (
            (B[0][0] + B[1][0]) * (-3 + cos_4T) * p1
            - 2 * A[0][0] * B[0][0] * (cos_4T - p1 * cos_2T)
            - 2 * A[0][0] * B[1][0] * (cos_4T - p1 * cos_2T)
            + 2 * sin_2T * (
                4 * (-B[0][0] + B[1][0]) * p4 * sin_2T ** 2
                + A[1][0] * B[1][0] * (-1 + cos_2T * p1 - 2 * p4 * sin_4T)
                - A[1][0] * B[0][0] * (-1 + cos_2T * p1 + 2 * p4 * sin_4T)
            )
        )
    )

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.dual)

sdp.write_to_file("num_level0m.csv")

1.082860684436476


## $T_{1+A+B+AA'B}$

In [8]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)

    # AAB
    for a in Aflat:
        for a2 in Aflat:
            for b in Bflat:
                monos += [a*a2*b]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()   # extra monomials

# Compute quantum value of Bell expression at (p1, p4) for a given Theta
s = np.sqrt(2)
Theta = np.pi/4
p1 = 1 - 1/np.sqrt(2)
p4 = 0

tan_2T = np.tan(2 * Theta)
sin_2T = np.sin(2 * Theta)
cos_2T = np.cos(2 * Theta)
cos_4T = np.cos(4 * Theta)
sin_4T = np.sin(4 * Theta)

obj = 1/16 * (
        4 * (s * (B[0][0] + B[1][0]) / (tan_2T * sin_2T) + 4 * p1 * A[0][0] + 8 * p4 * A[1][0]) +
        (4 / s) / (sin_2T ** 2) * (
            (B[0][0] + B[1][0]) * (-3 + cos_4T) * p1
            - 2 * A[0][0] * B[0][0] * (cos_4T - p1 * cos_2T)
            - 2 * A[0][0] * B[1][0] * (cos_4T - p1 * cos_2T)
            + 2 * sin_2T * (
                4 * (-B[0][0] + B[1][0]) * p4 * sin_2T ** 2
                + A[1][0] * B[1][0] * (-1 + cos_2T * p1 - 2 * p4 * sin_4T)
                - A[1][0] * B[0][0] * (-1 + cos_2T * p1 + 2 * p4 * sin_4T)
            )
        )
    )

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.dual)

sdp.write_to_file("num_level0m.csv")

1.0857864355065228


## $T_{1 + A + B + AB + ABB'}$

Without using $AB$ the bound it's not saturated! This means that we need to use $AB$!

In [ ]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)

    #AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]

    # ABB
    for a in Aflat:
        for b in Bflat:
            for b2 in Bflat:
                monos += [a*b*b2]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()   # extra monomials

# Compute quantum value of Bell expression at (p1, p4) for a given Theta
s = np.sqrt(2)
Theta = np.pi/4
p1 = (1 - 1/np.sqrt(2))
p4 = 0

tan_2T = np.tan(2 * Theta)
sin_2T = np.sin(2 * Theta)
cos_2T = np.cos(2 * Theta)
cos_4T = np.cos(4 * Theta)
sin_4T = np.sin(4 * Theta)

obj = 1/16 * (
        4 * (s * (B[0][0] + B[1][0]) / (tan_2T * sin_2T) + 4 * p1 * A[0][0] + 8 * p4 * A[1][0]) +
        (4 / s) / (sin_2T ** 2) * (
            (B[0][0] + B[1][0]) * (-3 + cos_4T) * p1
            - 2 * A[0][0] * B[0][0] * (cos_4T - p1 * cos_2T)
            - 2 * A[0][0] * B[1][0] * (cos_4T - p1 * cos_2T)
            + 2 * sin_2T * (
                4 * (-B[0][0] + B[1][0]) * p4 * sin_2T ** 2
                + A[1][0] * B[1][0] * (-1 + cos_2T * p1 - 2 * p4 * sin_4T)
                - A[1][0] * B[0][0] * (-1 + cos_2T * p1 + 2 * p4 * sin_4T)
            )
        )
    )

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.primal, -sdp.dual, sdp.status)

sdp.write_to_file("num_level1+A+B+AB+ABB.csv")
sos = sdp.get_sos_decomposition(threshold=1e-6)

1.0008911035610857 1.0008910972213176 optimal


Maybe we are able to reach this point at this level of the hierarchy.

In [10]:
b = sdp.y_mat[0]
print(b)
np.savetxt("Gamma.csv", b, delimiter=",")

[[ 2.58068385e-01 -6.83405996e-02 -7.47075445e-09  9.41907658e-02
   9.41907657e-02 -1.00718564e-01 -1.00718573e-01 -8.50311630e-02
   8.50311710e-02  6.34502709e-03  6.34501877e-03  3.34946168e-02
  -3.34946149e-02]
 [-6.83405996e-02  7.52804850e-02 -6.29326341e-10 -4.11503657e-02
  -4.11503677e-02  3.62397458e-02  3.62397458e-02  1.12897086e-02
  -1.12897085e-02  1.32092339e-02  1.32092347e-02  9.87534813e-03
  -9.87534681e-03]
 [-7.47075445e-09 -6.29326341e-10  6.58783548e-02 -5.57792175e-02
   5.57792151e-02  8.00618781e-03 -8.00618111e-03  9.07929759e-03
   9.07929331e-03  6.36724965e-03 -6.36725300e-03 -1.48824786e-02
  -1.48824762e-02]
 [ 9.41907658e-02 -4.11503657e-02 -5.57792175e-02  1.17416911e-01
  -8.56157562e-03 -3.90530056e-02 -6.34502711e-03 -6.01468311e-02
   2.76538593e-02 -3.49077548e-02 -3.13144669e-13  3.59663091e-02
  -3.10196130e-13]
 [ 9.41907657e-02 -4.11503677e-02  5.57792151e-02 -8.56157562e-03
   1.17416922e-01 -6.34501875e-03 -3.90530040e-02 -2.76538612e-02


## $T_{1 + A + B + AA' + BB' + AB + ABB'}$

In [11]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)

    # AA
    for a in Aflat:
        for a2 in Aflat:
            monos += [a*a2]

    # BB
    for b in Bflat:
        for b2 in Bflat:
            monos += [b*b2]
    
    # AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]

    # ABB
    for a in Aflat:
        for b in Bflat:
            for b2 in Bflat:
                monos += [a*b*b2]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()   # extra monomials

# Compute quantum value of Bell expression at (p1, p4) for a given Theta
s = np.sqrt(2)
Theta = np.pi/4
p1 = 1 - 1/np.sqrt(2)
p4 = 0

tan_2T = np.tan(2 * Theta)
sin_2T = np.sin(2 * Theta)
cos_2T = np.cos(2 * Theta)
cos_4T = np.cos(4 * Theta)
sin_4T = np.sin(4 * Theta)

obj = 1/16 * (
        4 * (s * (B[0][0] + B[1][0]) / (tan_2T * sin_2T) + 4 * p1 * A[0][0] + 8 * p4 * A[1][0]) +
        (4 / s) / (sin_2T ** 2) * (
            (B[0][0] + B[1][0]) * (-3 + cos_4T) * p1
            - 2 * A[0][0] * B[0][0] * (cos_4T - p1 * cos_2T)
            - 2 * A[0][0] * B[1][0] * (cos_4T - p1 * cos_2T)
            + 2 * sin_2T * (
                4 * (-B[0][0] + B[1][0]) * p4 * sin_2T ** 2
                + A[1][0] * B[1][0] * (-1 + cos_2T * p1 - 2 * p4 * sin_4T)
                - A[1][0] * B[0][0] * (-1 + cos_2T * p1 + 2 * p4 * sin_4T)
            )
        )
    )

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.primal, -sdp.dual, sdp.status)
sdp.write_to_file("num_level1+A+B+AA+BB+AB+ABB.csv")

1.0000000011791748 0.9999999983624781 optimal


**Note:** One can obtain the same result with ```level=2``` and adding just monomials of the form $ABB'$. 

## $T_{1+A+B+AB+AA'+BB'+AA'B}$

In [12]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)

    # AA
    for a in Aflat:
        for a2 in Aflat:
            monos += [a*a2]

    # BB
    for b in Bflat:
        for b2 in Bflat:
            monos += [b*b2]
    
    # AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]

    # AAB
    for a in Aflat:
        for a2 in Aflat:
            for b in Bflat:
                monos += [a*a2*b]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()   # extra monomials

# Compute quantum value of Bell expression at (p1, p4) for a given Theta
s = np.sqrt(2)
Theta = np.pi/4
p1 = 1 - 1/np.sqrt(2)
p4 = 0

tan_2T = np.tan(2 * Theta)
sin_2T = np.sin(2 * Theta)
cos_2T = np.cos(2 * Theta)
cos_4T = np.cos(4 * Theta)
sin_4T = np.sin(4 * Theta)

obj = 1/16 * (
        4 * (s * (B[0][0] + B[1][0]) / (tan_2T * sin_2T) + 4 * p1 * A[0][0] + 8 * p4 * A[1][0]) +
        (4 / s) / (sin_2T ** 2) * (
            (B[0][0] + B[1][0]) * (-3 + cos_4T) * p1
            - 2 * A[0][0] * B[0][0] * (cos_4T - p1 * cos_2T)
            - 2 * A[0][0] * B[1][0] * (cos_4T - p1 * cos_2T)
            + 2 * sin_2T * (
                4 * (-B[0][0] + B[1][0]) * p4 * sin_2T ** 2
                + A[1][0] * B[1][0] * (-1 + cos_2T * p1 - 2 * p4 * sin_4T)
                - A[1][0] * B[0][0] * (-1 + cos_2T * p1 + 2 * p4 * sin_4T)
            )
        )
    )

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.primal, -sdp.dual, sdp.status)
sdp.write_to_file("num_level1+A+B+AA+BB+AB+AAB.csv")

1.0000000006526162 0.9999999993275814 optimal


## $T_{1+A+B+AB+AA'+BB'+ABB'+AA'B}$

In [13]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)

    # AA
    for a in Aflat:
        for a2 in Aflat:
            monos += [a*a2]

    # BB
    for b in Bflat:
        for b2 in Bflat:
            monos += [b*b2]
    
    # AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]

    # ABB
    for a in Aflat:
        for b in Bflat:
            for b2 in Bflat:
                monos += [a*b*b2]
    # AAB
    for a in Aflat:
        for a2 in Aflat:
            for b in Bflat:
                monos += [a*a2*b]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()    # extra monomials

# Compute quantum value of Bell expression at (p1, p4) for a given Theta
s = np.sqrt(2)
Theta = np.pi/4
p1 = 1 - 1/np.sqrt(2)
p4 = 0

tan_2T = np.tan(2 * Theta)
sin_2T = np.sin(2 * Theta)
cos_2T = np.cos(2 * Theta)
cos_4T = np.cos(4 * Theta)
sin_4T = np.sin(4 * Theta)

obj = 1/16 * (
        4 * (s * (B[0][0] + B[1][0]) / (tan_2T * sin_2T) + 4 * p1 * A[0][0] + 8 * p4 * A[1][0]) +
        (4 / s) / (sin_2T ** 2) * (
            (B[0][0] + B[1][0]) * (-3 + cos_4T) * p1
            - 2 * A[0][0] * B[0][0] * (cos_4T - p1 * cos_2T)
            - 2 * A[0][0] * B[1][0] * (cos_4T - p1 * cos_2T)
            + 2 * sin_2T * (
                4 * (-B[0][0] + B[1][0]) * p4 * sin_2T ** 2
                + A[1][0] * B[1][0] * (-1 + cos_2T * p1 - 2 * p4 * sin_4T)
                - A[1][0] * B[0][0] * (-1 + cos_2T * p1 + 2 * p4 * sin_4T)
            )
        )
    )

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.dual)
sdp.write_to_file("num_level1+A+B+AA+BB+AB+ABB+AAB.csv")

0.9999999907686064


## Victor's Bell expression, $T_{1+A+B+AB+ABB'}$

In [4]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)

    #AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]

    # ABB
    for a in Aflat:
        for b in Bflat:
            for b2 in Bflat:
                monos += [a*b*b2]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()    # extra monomials

# Compute quantum value of Bell expression at (p1, p4) for a given Theta
r0 = 1 - 1/np.sqrt(2)
r1 = 0

# Victor's Bell expression
obj = r0*((A[0][0] + A[1][0])/np.sqrt(2) - B[0][0]) + r1*((A[0][0] - A[1][0])/np.sqrt(2) - B[1][0]) + 1/(2*np.sqrt(2))*(A[0][0]*B[0][0] + A[1][0]*B[0][0] + A[0][0]*B[1][0] - A[1][0]*B[1][0])

sdp = ncp.SdpRelaxation(ops, verbose=1, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.primal, -sdp.dual, sdp.status)
sosvictor = sdp.get_sos_decomposition()

The problem has 4 noncommuting Hermitian variables
Calculating block structure...
Estimated number of SDP variables: 90
Generating moment matrix...
Reduced number of SDP variables: 27 27 (done: 101.11%, ETA 00:00:-0.0)

Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 27              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 91)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries      

We want to extract the dual solution in the monomial basis:

In [4]:
Gamma = sdp.x_mat[0]
Wmon = sdp.y_mat[0]
print(Wmon)

np.savetxt("Wmon.csv", Wmon, delimiter=",")
np.savetxt("Gamma.csv", Gamma, delimiter=",")

[[ 2.01690088e-01 -4.70151546e-02 -4.70150952e-02  7.32242701e-02
   2.33116392e-10 -8.23302791e-02 -6.02862652e-02 -8.23302206e-02
   6.02862070e-02 -4.07413137e-04  4.35468180e-03  4.07412623e-04
  -4.35462303e-03]
 [-4.70151546e-02  8.97906867e-02  8.84035154e-03 -7.81066671e-02
  -5.43192393e-02  4.18571525e-02  1.80198594e-02  1.52269742e-02
   8.61485540e-03  1.30811542e-02  5.10053686e-03 -1.30811489e-02
  -9.23365298e-03]
 [-4.70150952e-02  8.84035154e-03  8.97907460e-02 -7.81066650e-02
   5.43192984e-02  1.52269320e-02 -8.61485567e-03  4.18571107e-02
  -1.80198596e-02  1.30811481e-02  9.23367103e-03 -1.30811538e-02
  -5.10057960e-03]
 [ 7.32242701e-02 -7.81066671e-02 -7.81066650e-02  1.98379981e-01
   1.07251209e-09 -5.21813173e-02  4.05720622e-04 -5.21813178e-02
  -4.05719524e-04 -6.21711906e-02  1.14861721e-12  6.21711897e-02
   7.48866333e-13]
 [ 2.33116392e-10 -5.43192393e-02  5.43192984e-02  1.07251209e-09
   9.99297193e-02 -4.35298928e-03 -4.35691852e-03  4.35292993e-03


## Victor's Bell expression reverted, $T_{1+A+B+AB+ABB'}$

In [3]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)

    #AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]

    # ABB
    for a in Aflat:
        for b in Bflat:
            for b2 in Bflat:
                monos += [a*b*b2]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()    # extra monomials

# Compute quantum value of Bell expression at (p1, p4) for a given Theta
r0 = 1 - 1/np.sqrt(2)
r1 = 0

# Victor's Bell expression
obj = r0*((B[0][0] + B[1][0])/np.sqrt(2) - A[0][0]) + r1*((B[0][0] - B[1][0])/np.sqrt(2) - A[1][0]) + 1/(2*np.sqrt(2))*(A[0][0]*B[0][0] + A[1][0]*B[0][0] + A[0][0]*B[1][0] - A[1][0]*B[1][0])

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.primal, -sdp.dual, sdp.status)
sosvictor = sdp.get_sos_decomposition()

1.000891103574409 1.0008910972216816 optimal


## Victor's Bell expression reverted, $T_{1+A+B+AB+AA'B}$

In [4]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)

    #AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]

    # AAB
    for a in Aflat:
        for a2 in Aflat:
            for b in Bflat:
                monos += [a*a2*b]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()    # extra monomials

# Compute quantum value of Bell expression at (p1, p4) for a given Theta
r0 = 1 - 1/np.sqrt(2)
r1 = 0

# Victor's Bell expression
obj = r0*((B[0][0] + B[1][0])/np.sqrt(2) - A[0][0]) + r1*((B[0][0] - B[1][0])/np.sqrt(2) - A[1][0]) + 1/(2*np.sqrt(2))*(A[0][0]*B[0][0] + A[1][0]*B[0][0] + A[0][0]*B[1][0] - A[1][0]*B[1][0])

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.primal, -sdp.dual, sdp.status)
sosvictor = sdp.get_sos_decomposition()

1.0000000083827505 0.9999999939894086 optimal


## Checks on the Bell expression

In [ ]:
# Compute quantum value of Bell expression at (p1, p4) for a given Theta
s = np.sqrt(2)
Theta = np.pi/4
# p1 = - r0
p1 = -(1 - 1/np.sqrt(2))
# p4 = -1/2*r1
p4 = 0

tan_2T = np.tan(2 * Theta)
sin_2T = np.sin(2 * Theta)
cos_2T = np.cos(2 * Theta)
cos_4T = np.cos(4 * Theta)
sin_4T = np.sin(4 * Theta)

obj = 1/16 * (
        4 * (s * (B[0][0] + B[1][0]) / (tan_2T * sin_2T) + 4 * p1 * A[0][0] + 8 * p4 * A[1][0]) +
        (4 / s) / (sin_2T ** 2) * (
            (B[0][0] + B[1][0]) * (-3 + cos_4T) * p1
            - 2 * A[0][0] * B[0][0] * (cos_4T - p1 * cos_2T)
            - 2 * A[0][0] * B[1][0] * (cos_4T - p1 * cos_2T)
            + 2 * sin_2T * (
                4 * (-B[0][0] + B[1][0]) * p4 * sin_2T ** 2
                + A[1][0] * B[1][0] * (-1 + cos_2T * p1 - 2 * p4 * sin_4T)
                - A[1][0] * B[0][0] * (-1 + cos_2T * p1 + 2 * p4 * sin_4T)
            )
        )
    )

print(obj)

0.0625*(4*(8.65956056235493e-17*(B0 + B1) - 1.17157287525381*A0) + 2.82842712474619*(2.0*(1.0*A1*B0 - 1.0*A1*B1) + 1.17157287525381*(B0 + B1) + 2.0*A0*B0 + 2.0*A0*B1))
